In [1]:
import pandas as pd
import holidays
import seaborn as sns
import requests
import numpy as np
from time import sleep
from shutil import copyfile
import random
from scipy import stats

In [2]:
%matplotlib inline
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

In [3]:
# # To find out where the pyspark
# import findspark
# findspark.init()

# # Creating Spark Context
# from pyspark import SparkContext, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import UserDefinedFunction as udf
from pyspark.sql.types import StringType, IntegerType, DecimalType, DoubleType

In [4]:
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler, CountVectorizer
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline

In [5]:
"""SimpleApp.py"""
spark = SparkSession.builder.appName("FirstSparkApp").getOrCreate()

In [48]:
df = spark.read\
    .format("com.databricks.spark.csv")\
    .option("header", "true") \
    .option("mode", "DROPMALFORMED")\
    .option("inferSchema", "true")\
    .load("/home/arditto_trianggada3/Workspace/ds-ovo-test/dataset/trip_merge_4_fe_all.csv")\
    .filter("pickup_datetime < '2013-04-08 '")

In [9]:
df1 = df.limit(10).toPandas()
df1.head(5)

,medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount,pickup_longitude_bin,pickup_latitude_bin,dropoff_longitude_bin,dropoff_latitude_bin,pickup_addr_zipcode,pickup_addr_place,pickup_addr_neighborhood,pickup_addr_sublocal,pickup_addr_local,pickup_addr_administrative_area_level_2,dropoff_addr_zipcode,dropoff_addr_place,dropoff_addr_neighborhood,dropoff_addr_sublocal,dropoff_addr_local,dropoff_addr_administrative_area_level_2,pickup_to_dropoff_zipcode,pickup_to_dropoff_place,pickup_to_dropoff_neighborhood,pickup_to_dropoff_sublocal,pickup_to_dropoff_local,pickup_to_dropoff_administrative_area_level_2,pickup_month,pickup_dayofweek,pickup_hour,dropoff_month,dropoff_dayofweek,dropoff_hour,is_public_holiday,is_weekend,is_holiday,trip_night,trip_late_night,trip_dist_euclidean,trip_dist_manhattan
0,91F6EB84975BBC867E32CB113C7C2CD5,AD8751110E6292079EB10EB9481FE1A6,CMT,1,N,2013-04-04 18:47:45,2013-04-04 19:00:25,1,759,2.5,-73.957855,40.765320,-73.976273,40.785648,CRD,11.0,1.0,0.5,2.50,0.0,15.00,-73.955,40.770,-73.975,40.790,10065.0,E 67th St,None,Manhattan,New York,New York County,10024.0,W 83rd St,None,Manhattan,New York,New York County,10065.0_10024.0,E 67th St_W 83rd St,nan_nan,Manhattan_Manhattan,New York_New York,New York County_New York County,4,Thursday,18,4,Thursday,19,0,0,0,1,0,0.027431,0.038746
1,C1B9DA774DC2BBC6DE27CE994E7F44A0,E1B595FD55E4C82C1E213EB17438107A,CMT,1,N,2013-04-04 17:59:50,2013-04-04 18:21:48,1,1318,3.6,-73.982880,40.754990,-74.009186,40.715374,CRD,16.5,1.0,0.5,3.60,0.0,21.60,-73.980,40.755,-74.005,40.720,10036.0,Verizon Plaza,None,Manhattan,New York,New York County,10007.0,Warren St,None,Manhattan,New York,New York County,10036.0_10007.0,Verizon Plaza_Warren St,nan_nan,Manhattan_Manhattan,New York_New York,New York County_New York County,4,Thursday,17,4,Thursday,18,0,0,0,1,0,0.047555,0.065922
2,9BA84250355AB3FC031C9252D395BF8A,16BB0D96A0DCC853AEC7F55C8D6C71E0,CMT,1,N,2013-04-04 18:12:01,2013-04-04 18:25:24,1,799,1.9,-73.978119,40.763451,-73.955666,40.776642,CRD,10.0,1.0,0.5,3.45,0.0,14.95,-73.975,40.765,-73.955,40.780,10019.0,7th Ave,None,Manhattan,New York,New York County,10028.0,E 81st St,None,Manhattan,New York,New York County,10019.0_10028.0,7th Ave_E 81st St,nan_nan,Manhattan_Manhattan,New York_New York,New York County_New York County,4,Thursday,18,4,Thursday,18,0,0,0,1,0,0.026041,0.035644
3,205A696DF62AD03C88DA8C5EC5248639,579C41EA5EC846F8B641A42F9EE3E855,CMT,1,N,2013-04-04 20:12:57,2013-04-04 20:29:55,1,1017,3.6,-74.006371,40.744755,-73.961662,40.761082,CRD,15.0,0.5,0.5,3.20,0.0,19.20,-74.005,40.745,-73.960,40.765,10011.0,W 19th St,None,Manhattan,New York,New York County,10022.0,E 60th St,None,Manhattan,New York,New York County,10011.0_10022.0,W 19th St_E 60th St,nan_nan,Manhattan_Manhattan,New York_New York,New York County_New York County,4,Thursday,20,4,Thursday,20,0,0,0,1,1,0.047597,0.061036
4,EE75E5927D00739AC342810C336A825E,1B4E92431F9DA4D49874EC76E769E874,CMT,1,N,2013-04-05 02:48:11,2013-04-05 02:51:21,2,189,0.7,-73.985191,40.754932,-73.990776,40.747997,CRD,4.5,0.5,0.5,1.10,0.0,6.60,-73.985,40.755,-73.990,40.750,10036.0,Verizon Plaza,None,Manhattan,New York,New York County,10120.0,West 33rd Street,None,Manhattan,New York,New York County,10036.0_10120.0,Verizon Plaza_West 33rd Street,nan_nan,Manhattan_Manhattan,New York_New York,New York County_New York County,4,Friday,2,4,Friday,2,0,0,0,0,1,0.008904,0.012520


In [49]:
df.printSchema()

root
 |-- medallion: string (nullable = true)
 |-- hack_license: string (nullable = true)
 |-- vendor_id: string (nullable = true)
 |-- rate_code: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_time_in_secs: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- surcharge: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- pickup_longitude_bin: double (nullable 

In [50]:
df.createOrReplaceTempView("tbl_df")

In [51]:
# df['trip_dist_euclidean'] = df.apply(lambda x: (np.abs(x['pickup_latitude']-x['dropoff_latitude']) ** 2 + np.abs(x['pickup_longitude']-x['dropoff_longitude'])  ** 2) ** 0.5, axis=1)


In [52]:
df_agg = spark.sql("""
SELECT
    t0.*,
    (unix_timestamp(pickup_datetime) - unix_timestamp(prev_dropoff_datetime)) pickup_time_in_secs,
    SQRT(POW(pickup_longitude-prev_dropoff_longitude,2) + POW(pickup_latitude-prev_dropoff_latitude,2)) pickup_dist_euclidean
    
FROM (
    SELECT
        hack_license,

        passenger_count,
        trip_time_in_secs,
        trip_distance,
        pickup_longitude,
        pickup_latitude,
        dropoff_longitude,
        dropoff_latitude,
        fare_amount,
        tip_amount,
        surcharge,
        tolls_amount,
        pickup_addr_zipcode,
        pickup_addr_local,
        dropoff_addr_zipcode,
        dropoff_addr_local,
        pickup_to_dropoff_zipcode,
        pickup_to_dropoff_local,
        pickup_dayofweek,
        pickup_hour,
        dropoff_dayofweek,
        dropoff_hour,
        trip_late_night,
        trip_night,
        trip_dist_euclidean,

        pickup_datetime,
        dropoff_datetime,

        --LAG(pickup_longitude) OVER (PARTITION BY hack_license,dropoff_dayofweek ORDER BY pickup_datetime) prev_pickup_longitude,
        --LAG(pickup_latitude) OVER (PARTITION BY hack_license,dropoff_dayofweek ORDER BY pickup_datetime) prev_pickup_latitude,
        LAG(dropoff_longitude) OVER (PARTITION BY hack_license,dropoff_dayofweek ORDER BY pickup_datetime) prev_dropoff_longitude,
        LAG(dropoff_latitude) OVER (PARTITION BY hack_license,dropoff_dayofweek ORDER BY pickup_datetime) prev_dropoff_latitude,
        
        --LAG(pickup_datetime) OVER (PARTITION BY hack_license,dropoff_dayofweek ORDER BY pickup_datetime) prev_pickup_datetime,
        LAG(dropoff_datetime) OVER (PARTITION BY hack_license,dropoff_dayofweek ORDER BY pickup_datetime) prev_dropoff_datetime

        
    FROM tbl_df
) t0

""")

In [53]:
df_agg.createOrReplaceTempView("tbl_df_agg")

In [23]:
df1 = df_agg.limit(10).toPandas()
df1

,hack_license,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,fare_amount,tip_amount,surcharge,tolls_amount,pickup_addr_zipcode,pickup_addr_local,dropoff_addr_zipcode,dropoff_addr_local,pickup_to_dropoff_zipcode,pickup_to_dropoff_local,pickup_dayofweek,pickup_hour,dropoff_dayofweek,dropoff_hour,trip_late_night,trip_night,trip_dist_euclidean,pickup_datetime,dropoff_datetime,prev_dropoff_longitude,prev_dropoff_latitude,prev_dropoff_datetime,pickup_time_in_secs,pickup_dist_euclidean
0,001EEDEA00E57988EF5213BCDD0D6B7C,1,608,6.2,-73.873131,40.774296,-73.814491,40.720524,18.5,3.00,0.0,0.0,11371.0,None,11367.0,None,11371.0_11367.0,nan_nan,Wednesday,10,Wednesday,10,0,0,0.079562,2013-04-03 10:19:56,2013-04-03 10:30:05,NaN,NaN,NaT,NaN,NaN
1,001EEDEA00E57988EF5213BCDD0D6B7C,1,2376,18.5,-73.863091,40.769287,-73.975929,40.604633,52.5,10.60,0.0,0.0,11371.0,None,11223.0,None,11371.0_11223.0,nan_nan,Wednesday,10,Wednesday,11,0,0,0.199608,2013-04-03 10:46:48,2013-04-03 11:26:25,-73.814491,40.720524,2013-04-03 10:30:05,1003.0,0.068846
2,001EEDEA00E57988EF5213BCDD0D6B7C,1,1084,2.9,-74.003815,40.713421,-74.007156,40.751408,14.0,2.00,0.0,0.0,10007.0,New York,10001.0,New York,10007.0_10001.0,New York_New York,Wednesday,12,Wednesday,13,0,0,0.038134,2013-04-03 12:55:11,2013-04-03 13:13:16,-73.975929,40.604633,2013-04-03 11:26:25,5326.0,0.112305
3,001EEDEA00E57988EF5213BCDD0D6B7C,1,1606,2.3,-74.006805,40.751301,-73.972954,40.750275,17.0,4.37,0.0,0.0,10001.0,New York,10016.0,New York,10001.0_10016.0,New York_New York,Wednesday,13,Wednesday,13,0,0,0.033867,2013-04-03 13:13:53,2013-04-03 13:40:40,-74.007156,40.751408,2013-04-03 13:13:16,37.0,0.000367
4,001EEDEA00E57988EF5213BCDD0D6B7C,1,566,2.1,-73.972473,40.749878,-73.993927,40.735176,9.5,2.00,0.0,0.0,10017.0,New York,10011.0,New York,10017.0_10011.0,New York_New York,Wednesday,13,Wednesday,13,0,0,0.026008,2013-04-03 13:41:48,2013-04-03 13:51:15,-73.972954,40.750275,2013-04-03 13:40:40,68.0,0.000624
5,001EEDEA00E57988EF5213BCDD0D6B7C,1,1521,3.2,-74.006493,40.738701,-73.969131,40.762611,17.0,3.50,0.0,0.0,10014.0,New York,10065.0,New York,10014.0_10065.0,New York_New York,Wednesday,13,Wednesday,14,0,0,0.044358,2013-04-03 13:57:27,2013-04-03 14:22:49,-73.993927,40.735176,2013-04-03 13:51:15,372.0,0.013051
6,001EEDEA00E57988EF5213BCDD0D6B7C,1,320,1.1,-73.968781,40.758728,-73.958946,40.772350,6.0,0.00,0.0,0.0,10022.0,New York,10021.0,New York,10022.0_10021.0,New York_New York,Wednesday,14,Wednesday,14,0,0,0.016801,2013-04-03 14:25:40,2013-04-03 14:31:01,-73.969131,40.762611,2013-04-03 14:22:49,171.0,0.003899
7,001EEDEA00E57988EF5213BCDD0D6B7C,2,311,0.5,-73.969193,40.755997,-73.963997,40.756920,5.0,0.00,0.0,0.0,10017.0,New York,10022.0,New York,10017.0_10022.0,New York_New York,Wednesday,14,Wednesday,14,0,0,0.005277,2013-04-03 14:43:58,2013-04-03 14:49:09,-73.958946,40.772350,2013-04-03 14:31:01,777.0,0.019298
8,001EEDEA00E57988EF5213BCDD0D6B7C,1,430,0.5,-73.968147,40.757080,-73.974915,40.755508,6.0,0.00,0.0,0.0,10017.0,New York,10029.0,New York,10017.0_10029.0,New York_New York,Wednesday,14,Wednesday,15,0,0,0.006948,2013-04-03 14:55:05,2013-04-03 15:02:16,-73.963997,40.756920,2013-04-03 14:49:09,356.0,0.004153
9,001EEDEA00E57988EF5213BCDD0D6B7C,4,920,1.8,-73.987160,40.747948,-73.974762,40.757683,11.0,0.00,0.0,0.0,10018.0,New York,10022.0,New York,10018.0_10022.0,New York_New York,Wednesday,15,Wednesday,15,0,0,0.015763,2013-04-03 15:14:24,2013-04-03 15:29:44,-73.974915,40.755508,2013-04-03 15:02:16,728.0,0.014391


In [54]:
df_agg1 = spark.sql("""
    SELECT
        hack_license,
        
        SUM(1) trip_num,
        
        MIN(passenger_count) passenger_count_min,
        PERCENTILE_APPROX(passenger_count,0.5) passenger_count_median,
        MAX(passenger_count) passenger_count_max,
        
        MIN(pickup_longitude) pickup_longitude_min,
        MIN(pickup_latitude) pickup_latitude_min,
        MAX(pickup_longitude) pickup_longitude_max,
        MAX(pickup_latitude) pickup_latitude_max,
        
        MIN(dropoff_longitude) dropoff_longitude_min,
        MIN(dropoff_latitude) dropoff_latitude_min,
        MAX(dropoff_longitude) dropoff_longitude_max,
        MAX(dropoff_latitude) dropoff_latitude_max,
        
        SUM(fare_amount) fare_amount_sum,
        
        SUM(tip_amount) tip_amount_sum,
        SUM(CASE WHEN tip_amount>0 THEN 1 ELSE 0 END) tip_amount_count,
        
        SUM(surcharge) surcharge_sum,
        SUM(CASE WHEN surcharge>0 THEN 1 ELSE 0 END) surcharge_count,
        
        CAST(COLLECT_LIST(pickup_to_dropoff_local) AS string) pickup_to_dropoff_local_list,
        CAST(COLLECT_SET(pickup_to_dropoff_local) AS string) pickup_to_dropoff_local_set,
        
        SUM(CASE WHEN pickup_hour==0  THEN 1 ELSE 0 END) pickup_hour_0 ,
        SUM(CASE WHEN pickup_hour==1  THEN 1 ELSE 0 END) pickup_hour_1 ,
        SUM(CASE WHEN pickup_hour==2  THEN 1 ELSE 0 END) pickup_hour_2 ,
        SUM(CASE WHEN pickup_hour==3  THEN 1 ELSE 0 END) pickup_hour_3 ,
        SUM(CASE WHEN pickup_hour==4  THEN 1 ELSE 0 END) pickup_hour_4 ,
        SUM(CASE WHEN pickup_hour==5  THEN 1 ELSE 0 END) pickup_hour_5 ,
        SUM(CASE WHEN pickup_hour==6  THEN 1 ELSE 0 END) pickup_hour_6 ,
        SUM(CASE WHEN pickup_hour==7  THEN 1 ELSE 0 END) pickup_hour_7 ,
        SUM(CASE WHEN pickup_hour==8  THEN 1 ELSE 0 END) pickup_hour_8 ,
        SUM(CASE WHEN pickup_hour==9  THEN 1 ELSE 0 END) pickup_hour_9 ,
        SUM(CASE WHEN pickup_hour==10 THEN 1 ELSE 0 END) pickup_hour_10,
        SUM(CASE WHEN pickup_hour==11 THEN 1 ELSE 0 END) pickup_hour_11,
        SUM(CASE WHEN pickup_hour==12 THEN 1 ELSE 0 END) pickup_hour_12,
        SUM(CASE WHEN pickup_hour==13 THEN 1 ELSE 0 END) pickup_hour_13,
        SUM(CASE WHEN pickup_hour==14 THEN 1 ELSE 0 END) pickup_hour_14,
        SUM(CASE WHEN pickup_hour==15 THEN 1 ELSE 0 END) pickup_hour_15,
        SUM(CASE WHEN pickup_hour==17 THEN 1 ELSE 0 END) pickup_hour_17,
        SUM(CASE WHEN pickup_hour==18 THEN 1 ELSE 0 END) pickup_hour_18,
        SUM(CASE WHEN pickup_hour==19 THEN 1 ELSE 0 END) pickup_hour_19,
        SUM(CASE WHEN pickup_hour==20 THEN 1 ELSE 0 END) pickup_hour_20,
        SUM(CASE WHEN pickup_hour==21 THEN 1 ELSE 0 END) pickup_hour_21,
        SUM(CASE WHEN pickup_hour==22 THEN 1 ELSE 0 END) pickup_hour_22,
        SUM(CASE WHEN pickup_hour==23 THEN 1 ELSE 0 END) pickup_hour_23,
        
        SUM(trip_late_night) trip_late_night_sum,
        SUM(trip_night) trip_night_sum,
        
        MIN(trip_time_in_secs) trip_time_in_secs_min,
        PERCENTILE_APPROX(trip_time_in_secs,0.5) trip_time_in_secs_median,
        MAX(trip_time_in_secs) trip_time_in_secs_max,
        SUM(trip_time_in_secs) trip_time_in_secs_sum,
        
        MIN(trip_dist_euclidean) trip_dist_euclidean_min,
        PERCENTILE_APPROX(trip_dist_euclidean,0.5) trip_dist_euclidean_median,
        MAX(trip_dist_euclidean) trip_dist_euclidean_max,
        SUM(trip_dist_euclidean) trip_dist_euclidean_sum,
        
        MIN(pickup_time_in_secs) pickup_time_in_secs_min,
        PERCENTILE_APPROX(pickup_time_in_secs,0.5) pickup_time_in_secs_median,
        MAX(pickup_time_in_secs) pickup_time_in_secs_max,
        SUM(pickup_time_in_secs) pickup_time_in_secs_sum,
        
        MIN(pickup_dist_euclidean) pickup_dist_euclidean_min,
        PERCENTILE_APPROX(pickup_dist_euclidean,0.5) pickup_dist_euclidean_median,
        MAX(pickup_dist_euclidean) pickup_dist_euclidean_max,
        SUM(pickup_dist_euclidean) pickup_dist_euclidean_sum,
        
        MIN(pickup_hour) pickup_hour,
        PERCENTILE_APPROX(pickup_hour,0.5) pickup_hour_median,
        MAX(pickup_hour) pickup_hour_max,
        
        (unix_timestamp(LAST(dropoff_datetime)) - unix_timestamp(FIRST(pickup_datetime))) work_time_in_secs
        
    FROM tbl_df_agg
    GROUP BY hack_license,dropoff_dayofweek

""")

In [56]:
df_agg1.write\
    .option('overwrite','true')\
    .option('header','true')\
    .csv('/home/arditto_trianggada3/Workspace/ds-ovo-test/dataset/spark_df/df_agg1')

In [6]:
df_agg1 = spark.read\
    .format("com.databricks.spark.csv")\
    .option("header", "true") \
    .option("mode", "DROPMALFORMED")\
    .option("inferSchema", "true")\
    .load('/home/arditto_trianggada3/Workspace/ds-ovo-test/dataset/spark_df/df_agg1')\
    .limit(10000)

In [58]:
df1 = df_agg1.limit(10).toPandas()
df1

,hack_license,trip_num,passenger_count_min,passenger_count_median,passenger_count_max,pickup_longitude_min,pickup_latitude_min,pickup_longitude_max,pickup_latitude_max,dropoff_longitude_min,dropoff_latitude_min,dropoff_longitude_max,dropoff_latitude_max,fare_amount_sum,tip_amount_sum,tip_amount_count,surcharge_sum,surcharge_count,pickup_to_dropoff_local_list,pickup_to_dropoff_local_set,pickup_hour_0,pickup_hour_1,pickup_hour_2,pickup_hour_3,pickup_hour_4,pickup_hour_5,pickup_hour_6,pickup_hour_7,pickup_hour_8,pickup_hour_9,pickup_hour_10,pickup_hour_11,pickup_hour_12,pickup_hour_13,pickup_hour_14,pickup_hour_15,pickup_hour_17,pickup_hour_18,pickup_hour_19,pickup_hour_20,pickup_hour_21,pickup_hour_22,pickup_hour_23,trip_late_night_sum,trip_night_sum,trip_time_in_secs_min,trip_time_in_secs_median,trip_time_in_secs_max,trip_time_in_secs_sum,trip_dist_euclidean_min,trip_dist_euclidean_median,trip_dist_euclidean_max,trip_dist_euclidean_sum,pickup_time_in_secs_min,pickup_time_in_secs_median,pickup_time_in_secs_max,pickup_time_in_secs_sum,pickup_dist_euclidean_min,pickup_dist_euclidean_median,pickup_dist_euclidean_max,pickup_dist_euclidean_sum,pickup_hour,pickup_hour_median,pickup_hour_max,work_time_in_secs
0,001916B0A5713FF05D936EFF82EEBA02,17,1,1,1,-74.007591,40.716557,-73.865509,40.779381,-74.008041,40.681931,-73.889580,40.807598,241.0,30.88,11,13.5,17,"[nan_New York, New York_New York, New York_New...","[New York_New York, nan_New York, nan_nan]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,4,5,3,2,1,1,7,13,120,660,2280,13920,0.002960,0.024471,0.128095,0.735419,60,120,2460,9120,0.000208,0.001903,0.127522,0.335225,17,19,23,23040
1,006114F940CB87B3ABDCE9BF6DF6FCC4,6,1,1,1,-73.983192,40.744667,-73.954529,40.776268,-73.982628,40.744312,-73.950111,40.800922,46.5,7.60,4,3.0,6,"[New York_New York, New York_New York, New Yor...",[New York_New York],3,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,6,0,120,300,840,2520,0.006816,0.009512,0.031274,0.110856,60,60,5820,7860,0.000572,0.000871,0.045157,0.056383,0,0,23,10380
2,0080BF52B83308D6C765F4D18B409303,15,1,1,1,-74.006287,40.645226,-73.776749,40.761871,-74.007698,40.724792,-73.864960,40.770424,186.0,26.93,10,0.0,0,"[nan_New York, New York_New York, New York_New...","[New York_New York, nan_New York, New York_nan]",0,0,0,0,0,0,0,0,1,2,4,2,4,2,0,0,0,0,0,0,0,0,0,0,0,120,720,3540,12540,0.006830,0.015444,0.219996,0.561000,60,180,3240,6060,0.000161,0.003676,0.009685,0.059836,8,11,13,18600
3,00A1DE03204248F82D05A0849F7182FD,18,1,1,4,-74.008278,40.695362,-73.874474,40.779915,-74.026054,40.621658,-73.861061,40.840637,257.5,20.75,9,8.5,17,"[New York_New York, New York_nan, nan_nan, nan...","[New York_New York, nan_New York, nan_nan, New...",3,0,0,1,1,2,1,0,0,0,0,0,0,0,0,0,0,0,0,3,3,2,2,18,0,184,607,1631,13940,0.004393,0.027034,0.170511,0.861998,104,643,50694,72555,0.000004,0.011244,0.139851,0.583241,0,20,23,86504
4,00A84F2983BCE93E943137C590D631D7,14,1,1,1,-74.005539,40.708080,-73.872787,40.790768,-74.008263,40.695087,-73.902351,40.787533,121.0,16.70,7,8.5,14,"[New York_New York, New York_New York, New Yor...","[New York_New York, nan_nan, New York_nan]",3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,4,0,0,4,11,7,65,463,1538,7046,0.005352,0.023390,0.044567,0.313024,39,258,64767,78988,0.000202,0.003260,0.080266,0.203560,0,20,23,86040
5,00BC0B18D4FFF31102DFCEFD013D04E1,22,1,1,1,-74.014008,40.716816,-73.945984,40.803574,-74.010880,40.716198,-73.936417,40.849545,220.5,21.99,13,1.5,3,"[New York_New York, New York_New York, New Yor...",[New York_New York],0,0,0,0,0,4,3,2,3,4,3,1,2,0,0,0,0,0,0,0,0,0,0,7,0,139,533,2018,14632,0.003254,0.017924,0.113930,0.566266,41,346,1678,9920,0.000302,0.003912,0.055377,0.170409,5,8,12,24561
6,00F19A8714CF5D4DF54F881F598ED80E,22,1,1,1,-74.007095,40.714336,-73.865883,40.808338,-74.005058,40.672546,-73.914467,40.807373,274.0,43.01,13,14.0,22,"[New York_New York, New York_New York, New Yor...","[New York_New York, nan_New York, nan_nan, New...",3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,2,1,4,5,3,16,7,240,

In [7]:
df1 = df_agg1.toPandas()

In [8]:
df1.to_csv("./dataset/trip_agg_hack_license_daily.csv",index=False)